In [1]:
from transformers import AutoModel, AutoTokenizer

/Users/roach/inception/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
text = "what is huggingface transformers?"

bert = AutoModel.from_pretrained("bert-base-uncased")
bert_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
encoded_output = bert_tokenizer(text, return_tensors="pt")
bert_output = bert(**encoded_output)

gpt = AutoModel.from_pretrained("gpt2")
gpt_tokenizer = AutoTokenizer.from_pretrained("gpt2")
encoded_output = gpt_tokenizer(text, return_tensors="pt")
gpt_output = gpt(**encoded_output)

print(bert_output)
print(gpt_output)

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.3009,  0.0158,  0.0698,  ..., -0.3406,  0.5976,  0.5820],
         [-0.1109,  0.0754, -0.1906,  ...,  0.2970,  0.4278, -0.0391],
         [-0.5813, -0.0042,  0.4034,  ..., -0.2549,  0.2216,  0.8121],
         ...,
         [ 0.9971,  0.3301, -0.0688,  ..., -0.4873,  0.0168, -0.0345],
         [-0.2394, -0.0573, -0.5885,  ..., -0.0415,  0.3123, -0.0288],
         [ 0.7884,  0.4039,  0.0217,  ...,  0.3869, -0.4785, -0.4116]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-8.7247e-01, -3.1464e-01, -5.2733e-01,  7.3685e-01,  1.8460e-01,
         -1.8295e-01,  8.9985e-01,  2.9762e-01, -3.9314e-01, -9.9994e-01,
          7.1272e-02,  7.2618e-01,  9.7232e-01,  3.2968e-01,  9.1370e-01,
         -7.5043e-01, -3.5754e-01, -5.6530e-01,  2.8968e-01, -6.4040e-01,
          6.0199e-01,  9.9856e-01,  3.8675e-01,  2.9078e-01,  4.0431e-01,
          7.7010e-01, -7.6515e-01,  9.1830e-01,  9.4834e-01,  7.255

In [3]:
from transformers import AutoModel

model_id = 'klue/roberta-base'
# 이건 model 의 body 를 불러오는 메소드이다.
model = AutoModel.from_pretrained(model_id)

Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
from transformers import AutoModelForSequenceClassification

# 이건 분류 헤드가 포함되어 있는 모델임.
model_id = 'SamLowe/roberta-base-go_emotions'
classification_model = AutoModelForSequenceClassification.from_pretrained(model_id)

In [5]:
from transformers import AutoModelForSequenceClassification
model_id = 'klue/roberta-base'
classification_model = AutoModelForSequenceClassification.from_pretrained(model_id)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
from transformers import AutoTokenizer

model_id = 'klue/roberta-base'
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [7]:
text = "오늘 날씨가 좋아요."
tokenized = tokenizer(text, return_tensors="pt")
tokenized

{'input_ids': tensor([[   0, 3822, 5792, 2116, 5723, 2182,   18,    2]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}

In [8]:
tokenizer.convert_ids_to_tokens(tokenized['input_ids'].tolist()[0])

['[CLS]', '오늘', '날씨', '##가', '좋아', '##요', '.', '[SEP]']

In [9]:
tokenizer.decode(tokenized['input_ids'].tolist()[0])

'[CLS] 오늘 날씨가 좋아요. [SEP]'

In [10]:
tokenizer.decode(tokenized['input_ids'].tolist()[0], skip_special_tokens=True)

'오늘 날씨가 좋아요.'

In [11]:
from datasets import load_dataset

klue_tc_train = load_dataset("klue", "ynat", split="train")
klue_tc_eval = load_dataset("klue", "ynat", split="validation")
klue_tc_train

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Dataset({
    features: ['guid', 'title', 'label', 'url', 'date'],
    num_rows: 45678
})

In [12]:
klue_tc_train.features['label'].names

['IT과학', '경제', '사회', '생활문화', '세계', '스포츠', '정치']

In [13]:
klue_tc_train = klue_tc_train.remove_columns(['guid', 'url', 'date'])
klue_tc_eval = klue_tc_eval.remove_columns(['guid', 'url', 'date'])
klue_tc_train

Dataset({
    features: ['title', 'label'],
    num_rows: 45678
})

In [14]:
klue_tc_train.features['label'].int2str(1)

'경제'

In [15]:
klue_tc_label = klue_tc_train.features['label']

def make_str_label(batch):
    batch['label_str'] = klue_tc_label.int2str(batch['label'])

In [16]:
klue_tc_train = klue_tc_train.map(make_str_label, batched=True, batch_size=1000)
klue_tc_train[0]

Map: 100%|██████████| 45678/45678 [00:00<00:00, 2202078.30 examples/s]


{'title': '유튜브 내달 2일까지 크리에이터 지원 공간 운영', 'label': 3}

In [17]:
train_dataset = klue_tc_train.train_test_split(test_size=10000, shuffle=True, seed=42)['test']
dataset = klue_tc_eval.train_test_split(test_size=1000, shuffle=True, seed=42)
test_dataset = dataset['test']
validation_dataset = klue_tc_eval.train_test_split(test_size=1000, shuffle=True, seed=42)['test']

In [18]:
import torch
import numpy as np
from transformers import (
    Trainer,
    TrainingArguments,
    AutoModelForSequenceClassification,
    AutoTokenizer,
)

def tokenize_function(examples):
    return tokenizer(examples['title'], padding='max_length', truncation=True)

model_id = 'klue/roberta-base'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=len(train_dataset.features['label'].names))

train_dataset = train_dataset.map(tokenize_function, batched=True)
valid_dataset = validation_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 10000/10000 [00:00<00:00, 13159.62 examples/s]


In [19]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    evaluation_strategy='epoch',
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    push_to_hub=False,
)


def compute_metrics(p):
    logits, labels = p
    predictions = np.argmax(logits, axis=-1)
    return {'accuracy': (predictions == labels).mean()}

/Users/roach/inception/.venv/lib/python3.13/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [20]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

trainer.evaluate(test_dataset)

/var/folders/cv/y8g_j3056h5dw85l1b41k7400000gn/T/ipykernel_77440/143171837.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.529800,0.450205,0.863000


{'eval_loss': 0.45020490884780884,
 'eval_accuracy': 0.863,
 'eval_runtime': 25.9649,
 'eval_samples_per_second': 38.514,
 'eval_steps_per_second': 4.814,
 'epoch': 1.0}

In [36]:
outputs = model(tokenizer.encode("오늘 미국의 거시 경제는?", return_tensors="pt").to("mps"))
logits = outputs.logits  # logits 가져오기
predictions = torch.argmax(logits, dim=-1)  # 예측값 계산
answer = predictions.item()  # 단일 정수값으로 변환

# 레이블 변환
label = klue_tc_label.int2str(answer)
print(label)

세계
